In [1]:
#from Preprocess import train_dataset, val_dataset, test_dataset, amazon_training_image_paths, amazon_validation_image_paths, amazon_test_image_paths, batches

import os
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
from tensorflow.keras.layers import Conv2D, MaxPooling2D, concatenate, UpSampling2D, BatchNormalization, Activation, LeakyReLU, Dropout, Input, Lambda, Add, Conv2DTranspose, Concatenate, Reshape, Permute, Multiply, GlobalAveragePooling2D, Dense, Flatten

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, jaccard_score
import tensorflow.keras.backend as K
#from hyperparameters import alpha
K.set_image_data_format('channels_last')
from tensorflow.keras.models import Model

In [2]:
import _02c_read_datasets
from tensorflow.keras import callbacks

# -------------------- load data

augment = False
train_dataset, val_dataset, test_dataset = _02c_read_datasets.load_datasets(augmented = augment)

# ----------- create directories

out_dir = '../results/' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '_UNET_ATTN/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    os.makedirs(out_dir + '/plots')
    os.makedirs(out_dir + '/weights')
    os.makedirs(out_dir + '/predictions')

checkpoint = callbacks.ModelCheckpoint(
    filepath=out_dir+'weights/'+'model.{epoch:02d}-{val_loss:.4f}.h5',
    save_weights_only=True,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max', 
    verbose=1
)

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-04-27 07:11:57.294165: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-27 07:11:57.294519: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
def conv2d_block(input_tensor,
                 n_filters,
                 kernel_size=3,
                 batchnorm=True,
                 strides=1,
                 dilation_rate=1,
                 recurrent=1):

    # A wrapper of the Keras Conv2D block to serve as a building block for downsampling layers
    # Includes options to use batch normalization, dilation and recurrence

    conv = Conv2D(filters=n_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  kernel_initializer="he_normal",
                  padding="same",
                  dilation_rate=dilation_rate)(input_tensor)
    if batchnorm:
        conv = BatchNormalization()(conv)
    output = LeakyReLU(alpha=0.3)(conv)

    for _ in range(recurrent - 1):
        conv = Conv2D(filters=n_filters,
                      kernel_size=kernel_size,
                      strides=1,
                      kernel_initializer="he_normal",
                      padding="same",
                      dilation_rate=dilation_rate)(output)
        if batchnorm:
            conv = BatchNormalization()(conv)
        res = LeakyReLU(alpha=0.3)(conv)
        output = Add()([output, res])

    return output

def transpose_block(input_tensor,
                    skip_tensor,
                    n_filters,
                    kernel_size=3,
                    strides=1,
                    batchnorm=True,
                    recurrent=1):

    # A wrapper of the Keras Conv2DTranspose block to serve as a building block for upsampling layers

    shape_x = K.int_shape(input_tensor)
    shape_xskip = K.int_shape(skip_tensor)

    conv = Conv2DTranspose(filters=n_filters,
                           kernel_size=kernel_size,
                           padding='same',
                           strides=(shape_xskip[1] // shape_x[1],
                                    shape_xskip[2] // shape_x[2]),
                           kernel_initializer="he_normal")(input_tensor)
    conv = LeakyReLU(alpha=0.3)(conv)

    act = conv2d_block(conv,
                       n_filters=n_filters,
                       kernel_size=kernel_size,
                       strides=1,
                       batchnorm=batchnorm,
                       dilation_rate=1,
                       recurrent=recurrent)
    output = Concatenate(axis=3)([act, skip_tensor])
    return output

def expend_as(tensor, rep):

    # Anonymous lambda function to expand the specified axis by a factor of argument, rep.
    # If tensor has shape (512,512,N), lambda will return a tensor of shape (512,512,N*rep), if specified axis=2

    my_repeat = Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3),
                       arguments={'repnum': rep})(tensor)
    return my_repeat

def AttnGatingBlock(x, g, inter_shape):

    shape_x = K.int_shape(x)
    shape_g = K.int_shape(g)

    # Getting the gating signal to the same number of filters as the inter_shape
    phi_g = Conv2D(filters=inter_shape,
                   kernel_size=1,
                   strides=1,
                   padding='same')(g)

    # Getting the x signal to the same shape as the gating signal
    theta_x = Conv2D(filters=inter_shape,
                     kernel_size=3,
                     strides=(shape_x[1] // shape_g[1],
                              shape_x[2] // shape_g[2]),
                     padding='same')(x)

    # Element-wise addition of the gating and x signals
    # add_xg = Add([phi_g, theta_x])
    add_xg = Add()([phi_g, theta_x])

    add_xg = Activation('relu')(add_xg)

    # 1x1x1 convolution
    psi = Conv2D(filters=1, kernel_size=1, padding='same')(add_xg)
    psi = Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(psi)

    # Upsampling psi back to the original dimensions of x signal
    upsample_sigmoid_xg = UpSampling2D(size=(shape_x[1] // shape_sigmoid[1],
                                             shape_x[2] //
                                             shape_sigmoid[2]))(psi)

    # Expanding the filter axis to the number of filters in the original x signal
    upsample_sigmoid_xg = expend_as(upsample_sigmoid_xg, shape_x[3])

    # Element-wise multiplication of attention coefficients back onto original x signal
    attn_coefficients = Multiply()([upsample_sigmoid_xg, x])

    # Final 1x1x1 convolution to consolidate attention signal to original x dimensions
    output = Conv2D(filters=shape_x[3],
                    kernel_size=1,
                    strides=1,
                    padding='same')(attn_coefficients)
    output = BatchNormalization()(output)
    return output


def GatingSignal(input_tensor, batchnorm=True):

    # 1x1x1 convolution to consolidate gating signal into the required dimensions
    # Not required most of the time, unless another ReLU and batch_norm is required on gating signal

    shape = K.int_shape(input_tensor)
    conv = Conv2D(filters=shape[3],
                  kernel_size=1,
                  strides=1,
                  padding="same",
                  kernel_initializer="he_normal")(input_tensor)
    if batchnorm:
        conv = BatchNormalization()(conv)
    output = LeakyReLU(alpha=alpha)(conv)
    return output

In [4]:
def Attention_U_network(input_shape, n_filters=16, batchnorm=True):

    inputs = tf.keras.Input(input_shape)
    
    # encoder
    
    c0 = conv2d_block(inputs,
                 n_filters=n_filters,
                 kernel_size=3,
                 batchnorm=batchnorm,
                 strides=1,
                 dilation_rate=1,
                 recurrent=2) # 256x256x256
    
    p0 = MaxPooling2D((2,2))(c0)
    p0 = Dropout(0.4)(p0)
    
    c1 = conv2d_block(p0,
                 n_filters=n_filters * 2,
                 kernel_size=3,
                 batchnorm=batchnorm,
                 strides=1,
                 dilation_rate=1,
                 recurrent=2) # 128x128x128
    
    p1 = MaxPooling2D((2,2))(c1)
    p1 = Dropout(0.4)(p1)
    
    c2 = conv2d_block(p1,
                 n_filters=n_filters * 4,
                 kernel_size=3,
                 batchnorm=batchnorm,
                 strides=1,
                 dilation_rate=1,
                 recurrent=2) # 64x64x64
    
    p2 = MaxPooling2D((2,2))(c2)
    p2 = Dropout(0.4)(p2)

    c3 = conv2d_block(p2,
                 n_filters=n_filters * 8,
                 kernel_size=3,
                 batchnorm=batchnorm,
                 strides=1,
                 dilation_rate=1,
                 recurrent=2) # 32x32x32
    
    p3 = MaxPooling2D((2,2))(c3)
    p3 = Dropout(0.4)(p3)

    # bridge
    
    b0 = conv2d_block(p3,
                 n_filters=n_filters * 16,
                 kernel_size=3,
                 batchnorm=batchnorm,
                 strides=1,
                 dilation_rate=1,
                 recurrent=2) # 16x16x16

    # decoder

    attn0 = AttnGatingBlock(c3, b0, n_filters * 16)
    u0 = transpose_block(b0,
                         attn0,
                         n_filters=n_filters * 8,
                         batchnorm=batchnorm,
                         recurrent=2)  # 32x32x32

    attn1 = AttnGatingBlock(c2, u0, n_filters * 8)
    u1 = transpose_block(u0,
                         attn1,
                         n_filters=n_filters * 4,
                         batchnorm=batchnorm,
                         recurrent=2)  # 64x64x64

    attn2 = AttnGatingBlock(c1, u1, n_filters * 4)
    u2 = transpose_block(u1,
                         attn2,
                         n_filters=n_filters * 2,
                         batchnorm=batchnorm,
                         recurrent=2)  # 128x128x128

    u3 = transpose_block(u2,
                         c0,
                         n_filters=n_filters,
                         batchnorm=batchnorm,
                         recurrent=2)  # 256x256x256

    outputs = Conv2D(filters=1, kernel_size=1, strides=1,
                     activation='sigmoid')(u3)
    model = Model(inputs=inputs, outputs=[outputs])
    return model

In [5]:
input_shape = (256, 256, 4)
model = Attention_U_network(input_shape, batchnorm=True)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])
epochs = 20
history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[checkpoint])

Epoch 1/20


2023-04-27 07:12:02.980527: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-27 07:12:04.161505: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 27/125 [=====>........................] - ETA: 36s - loss: 0.2758 - accuracy: 0.8831

In [ ]:
# ---------------------- save results

# load best model
model.load_weights(out_dir + 'weights/'+'_.h5')

In [ ]:

# training and validation loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.savefig(out_dir + '/plots/' + 'loss.png')

In [ ]:

# training and validation accuracy
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.legend()
plt.savefig(out_dir + '/plots/' + 'accuracy.png')


In [ ]:

# save weights
# model.save(out_dir + '/weights/' + 'model.h5')

# save predictions
def visualize_predictions(index, test_dataset, out_dir, batches = 16):
    
    dir = "image_" + str(index)
    if not os.path.exists(out_dir + '/predictions/' + dir + '/'):
        os.makedirs(out_dir + '/predictions/' + dir + '/')
        os.makedirs(out_dir + '/predictions/' + dir + '/input_image')
        os.makedirs(out_dir + '/predictions/' + dir + '/ground_truth')
        os.makedirs(out_dir + '/predictions/' + dir + '/prediction')
        os.makedirs(out_dir + '/predictions/' + dir + '/prediction_binary')
    
    test_data_iter = iter(itertools.cycle(test_dataset))

    for i in range(index + 1):
        image_batch, label_batch = next(test_data_iter)

    wrapped_index = index % 16
    image = image_batch[wrapped_index].numpy()
    image_rgb = np.stack(
        (
            (image[:,:,0] - np.min(image[:,:,0])) * 255.0 / (np.max(image[:,:,0]) - np.min(image[:,:,0])),
            (image[:,:,1] - np.min(image[:,:,1])) * 255.0 / (np.max(image[:,:,1]) - np.min(image[:,:,1])),
            (image[:,:,2] - np.min(image[:,:,2])) * 255.0 / (np.max(image[:,:,2]) - np.min(image[:,:,2]))
        ),
        axis=-1
    ).astype(np.uint8)

    prediction = model.predict(np.expand_dims(image, axis=0))[0]
    plt.imsave(out_dir + '/predictions/' + dir + '/input_image/' + str(index) + '.png', image_rgb)

    ground_truth = label_batch[wrapped_index].numpy()
    plt.imsave(out_dir + '/predictions/' + dir + '/ground_truth/' + str(index) + '.png', np.squeeze(ground_truth), cmap='gray')

    plt.imsave(out_dir + '/predictions/' + dir + '/prediction/' + str(index) + '.png', np.squeeze(prediction), cmap='gray')

    prediction_binary = np.where(prediction > 0.5, 1, 0)
    plt.imsave(out_dir + '/predictions/' + dir + '/prediction_binary/' + str(index) + '.png', np.squeeze(prediction_binary), cmap='gray')

for i in range(80):
    visualize_predictions(i, test_dataset, out_dir)


In [ ]:

# ----------- save metrics

if augment:
    augmetation_settings = {
    "flip_left_right": 0,
    "flip_up_down": 0,
    "gaussian_blur": 0.2,
    "random_noise": 0.0,
    "random_brightness": 0.5,
    "random_contrast": 0.5}
else:
    augmetation_settings = None

batches = 16
shuffled = True

model_info = _02_evaluate_model.evaluate_model(
    "U-net without attention; final dataset", 
    test_dataset,
    model, 
    input_shape, 
    shuffled, 
    batches, 
    epochs, 
    augmentation_settings=augmetation_settings, 
    threshold=0.5
    )
df = pd.DataFrame(model_info)
df.to_csv(os.path.join(out_dir, 'metrics.csv'), index=False)